In [82]:
import numpy as np
from tensorflow import keras

In [83]:
def preprocess_images(dir, subset, image_size=(128, 128)):
    imgs = keras.utils.image_dataset_from_directory(
        dir,
        subset=subset,
        labels='inferred',
        label_mode='binary',
        class_names=None,
        color_mode='grayscale',
        batch_size=16,
        seed=313,
        image_size=image_size,
        shuffle=True,
        validation_split=0.1,
        interpolation='bilinear',
)

    return imgs

In [84]:
from matplotlib import pyplot as plt

# 

dir = 'datasets/brain_mri_scan_images'
train_ds = preprocess_images(dir, 'training')
test_ds = preprocess_images(dir, 'validation')

# Optional visualization to ensure the dataset imported properly

# class_names = train_ds.class_names
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     print(f'Labels = {int(labels[i])}')
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
#     plt.title(class_names[int(labels[i])] + str(int(labels[i])))
#     plt.axis("off")



Found 216 files belonging to 2 classes.
Using 195 files for training.
Found 216 files belonging to 2 classes.
Using 21 files for validation.


In [85]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, RandomFlip
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input, concatenate
from keras.layers.core import Dropout, Lambda

inputs = Input((128, 128, 1))

s = Lambda(lambda x: x / 255) (inputs)

pp_1 = keras.layers.RandomFlip('horizontal') (s)



c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pp_1)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

f1 = Flatten() (c9)

outputs = Dense(1, activation="sigmoid") (f1)

model = Model(inputs=[inputs], outputs=[outputs])

model.compile(optimizer='adam', loss='binary_crossentropy')



In [86]:
filepath = "model.h5"

earlystopper = EarlyStopping(patience=5, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')

callbacks_list = [earlystopper, checkpoint]


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 lambda_13 (Lambda)             (None, 128, 128, 1)  0           ['input_14[0][0]']               
                                                                                                  
 random_flip_13 (RandomFlip)    (None, 128, 128, 1)  0           ['lambda_13[0][0]']              
                                                                                                  
 conv2d_234 (Conv2D)            (None, 128, 128, 16  160         ['random_flip_13[0][0]']  

In [87]:
batch_size = 16
epochs = 40

history = model.fit(train_ds, validation_data=test_ds, batch_size=batch_size, epochs=epochs, 
                    callbacks=callbacks_list)

Epoch 1/40


13/13 [==============================] - ETA: 0s - loss: 10.4174 - accuracy: 0.5641
Epoch 1: val_loss improved from inf to 16.80600, saving model to model.h5
13/13 [==============================] - 2s 72ms/step - loss: 10.4174 - accuracy: 0.5641 - val_loss: 16.8060 - val_accuracy: 0.4286
Epoch 2/40
13/13 [==============================] - ETA: 0s - loss: 3.3939 - accuracy: 0.6615
Epoch 2: val_loss improved from 16.80600 to 3.16427, saving model to model.h5
13/13 [==============================] - 1s 55ms/step - loss: 3.3939 - accuracy: 0.6615 - val_loss: 3.1643 - val_accuracy: 0.7143
Epoch 3/40
13/13 [==============================] - ETA: 0s - loss: 1.4843 - accuracy: 0.7590
Epoch 3: val_loss improved from 3.16427 to 2.00232, saving model to model.h5
13/13 [==============================] - 1s 56ms/step - loss: 1.4843 - accuracy: 0.7590 - val_loss: 2.0023 - val_accuracy: 0.7143
Epoch 4/40
13/13 [==============================] - ETA: 0s - loss: 0.2880 - accuracy: 0.9128
Epoch 4: val_

In [101]:
import os
import re

# All this code is to check if the new, best model beats the existing best in both val_loss and accuracy

mvl_index = history.history['val_loss'].index(min(history.history['val_loss']))
min_val_loss = round(history.history['val_loss'][mvl_index], 4)
val_accuracy = round(history.history['val_accuracy'][mvl_index], 4)

print(history.history['val_loss'])
print(f'Index = {mvl_index}, Val_Loss = {min_val_loss}, Val_Acc = {val_accuracy}')

for file in os.listdir('.'):
    if file.endswith('.h5') and file.startswith('model-best'):
        best_val_loss, best_val_accuracy = re.findall('\d+\.\d+', file)
        print(best_val_loss, best_val_accuracy)

        


[16.806001663208008, 3.164273977279663, 2.002323865890503, 2.0542538166046143, 0.4147034287452698, 1.8829033374786377, 2.273947238922119, 0.9930100440979004, 1.8238283395767212, 1.113222599029541]
Index = 4, Val_Loss = 0.4147, Val_Acc = 0.8571
0.4147 0.8571
